# DATA EDA

### Imports

In [66]:
#Imports
import pandas as pd
import numpy as np
from imblearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer


### Dataframes Imports 

In [32]:
#importing 1st transactions
transactions_0_df= pd.read_csv("../raw_data/transactions.csv")
#Importing Transactions data
transactions_df= pd.read_csv("../raw_data/transactions_v2.csv")
#Importing user logs data
user_logs_df = pd.read_csv("../raw_data/user_logs_v2.csv")
#importing members data
members_df = pd.read_csv("../raw_data/members_v3.csv")
# import first training dataset
train_0_df= pd.read_csv("../raw_data/train.csv")
# import training dataset
train_df = pd.read_csv("../raw_data/train_v2.csv")
# import test dataset
test_df = pd.read_csv("../raw_data/sample_submission_zero.csv")
# import test dataset 2
test_2_df = pd.read_csv("../raw_data/sample_submission_v2.csv")

### Building trendline

In [33]:
#concat vertically
transactions_df_concat = pd.concat([transactions_0_df, transactions_df], axis=0)

#drop duplicates
transactions_df_global = transactions_df_concat.drop_duplicates(keep="first")

In [34]:
#concat vertically
test_df_concat = pd.concat([test_df, test_2_df], axis=0)

#drop duplicates
test_df = test_df_concat.drop_duplicates(keep="first")


In [35]:
#merge with transaction data
transactions_test_df = pd.merge(test_df, transactions_df_global, on='msno', how='left')
transactions_test_df

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0,38,30,149,149,0,20170228,20170330,0
1,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0,31,0,0,149,1,20150331,20150430,0
2,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0,31,0,0,149,1,20150630,20150731,0
3,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0,31,30,149,149,1,20150902,20150901,1
4,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0,31,0,0,149,1,20150831,20150930,0
...,...,...,...,...,...,...,...,...,...,...
17130170,r+Eb1hNptQDU/Wa0E+Myg/4+K7XqckAvlEU6xY2r9CQ=,0,41,30,99,99,1,20170201,20170301,0
17130171,r+Eb1hNptQDU/Wa0E+Myg/4+K7XqckAvlEU6xY2r9CQ=,0,41,30,99,99,1,20160801,20160901,0
17130172,r+Eb1hNptQDU/Wa0E+Myg/4+K7XqckAvlEU6xY2r9CQ=,0,41,30,99,99,1,20160901,20161001,0
17130173,r+Eb1hNptQDU/Wa0E+Myg/4+K7XqckAvlEU6xY2r9CQ=,0,41,30,99,99,1,20161201,20170101,0


In [36]:
#convert to datetime
transactions_test_df.loc[:,'transaction_date'] = pd.to_datetime(transactions_test_df['transaction_date'], format='%Y%m%d')
transactions_test_df.loc[:,'membership_expire_date'] = pd.to_datetime(transactions_test_df['membership_expire_date'], format='%Y%m%d')

In [37]:
# Sort the DataFrame by 'msno' and 'transaction_date'
transactions_test_df = transactions_test_df.sort_values(by=['msno', 'transaction_date'], ascending=[True, True],)

In [38]:
#compute previous period churn
transactions_test_df['transaction_date_-1'] = transactions_test_df.groupby('msno')['transaction_date'].shift(+1)
transactions_test_df['membership_expire_date_-1'] = transactions_test_df.groupby('msno')['membership_expire_date'].shift(+1)
transactions_test_df['period_0'] = transactions_test_df['membership_expire_date_-1']-transactions_test_df['transaction_date']
transactions_test_df['period_0_churn'] = np.where(transactions_test_df['period_0'] <= pd.Timedelta(days=-30), 1, np.where(transactions_test_df['period_0'].isna(), np.nan, 0))

transactions_test_df['transaction_date_-2'] = transactions_test_df.groupby('msno')['transaction_date'].shift(+2)
transactions_test_df['membership_expire_date_-2'] = transactions_test_df.groupby('msno')['membership_expire_date'].shift(+2)
transactions_test_df['period_-1'] = transactions_test_df['membership_expire_date_-2']-transactions_test_df['transaction_date_-1']
transactions_test_df['period_-1_churn'] = np.where(transactions_test_df['period_-1'] <= pd.Timedelta(days=-30), 1, np.where(transactions_test_df['period_-1'].isna(), np.nan, 0))


transactions_test_df['transaction_date_-3'] = transactions_test_df.groupby('msno')['transaction_date'].shift(+3)
transactions_test_df['membership_expire_date_-3'] = transactions_test_df.groupby('msno')['membership_expire_date'].shift(+3)
transactions_test_df['period_-2'] = transactions_test_df['membership_expire_date_-3']-transactions_test_df['transaction_date_-2']
transactions_test_df['period_-2_churn'] = np.where(transactions_test_df['period_-2'] <= pd.Timedelta(days=-30), 1, np.where(transactions_test_df['period_-2'].isna(), np.nan, 0))


transactions_test_df['transaction_date_-4'] = transactions_test_df.groupby('msno')['transaction_date'].shift(+4)
transactions_test_df['membership_expire_date_-4'] = transactions_test_df.groupby('msno')['membership_expire_date'].shift(+4)
transactions_test_df['period_-3'] = transactions_test_df['membership_expire_date_-4']-transactions_test_df['transaction_date_-3']
transactions_test_df['period_-3_churn'] = np.where(transactions_test_df['period_-3'] <= pd.Timedelta(days=-30), 1, np.where(transactions_test_df['period_-3'].isna(), np.nan, 0))


transactions_test_df['transaction_date_-5'] = transactions_test_df.groupby('msno')['transaction_date'].shift(+5)
transactions_test_df['membership_expire_date_-5'] = transactions_test_df.groupby('msno')['membership_expire_date'].shift(+5)
transactions_test_df['period_-4'] = transactions_test_df['membership_expire_date_-5']-transactions_test_df['transaction_date_-4']
transactions_test_df['period_-4_churn'] = np.where(transactions_test_df['period_-4'] <= pd.Timedelta(days=-30), 1, np.where(transactions_test_df['period_-4'].isna(), np.nan, 0))


transactions_test_df['transaction_date_-6'] = transactions_test_df.groupby('msno')['transaction_date'].shift(+6)
transactions_test_df['membership_expire_date_-6'] = transactions_test_df.groupby('msno')['membership_expire_date'].shift(+6)
transactions_test_df['period_-5'] = transactions_test_df['membership_expire_date_-6']-transactions_test_df['transaction_date_-5']
transactions_test_df['period_-5_churn'] = np.where(transactions_test_df['period_-5'] <= pd.Timedelta(days=-30), 1, np.where(transactions_test_df['period_-5'].isna(), np.nan, 0))




In [39]:
#only keeping rows with the latest transactions 
latest_transaction_indexes = transactions_test_df.groupby('msno')['transaction_date'].idxmax()
latest_transactions_with_churn = transactions_test_df.loc[latest_transaction_indexes]

In [40]:
# removing unnecessary columns
trendline_df = latest_transactions_with_churn.drop(axis=1, columns=[
    'transaction_date_-1',
    'membership_expire_date_-1',
    'period_0',
    'transaction_date_-2',
    'membership_expire_date_-2',
    'period_-1',
    'transaction_date_-3',
    'membership_expire_date_-3',
    'period_-2',
    'transaction_date_-4',
    'membership_expire_date_-4',
    'period_-3',
    'transaction_date_-5',
    'membership_expire_date_-5',
    'period_-4',
    'transaction_date_-6',
    'membership_expire_date_-6',
    'period_-5',
    'is_churn',
    'payment_method_id',
    'payment_plan_days',
    'plan_list_price',
    'actual_amount_paid',
    'is_auto_renew',
    'transaction_date',
    'membership_expire_date',
    'is_cancel'
])
                                                                        

### Transactions Data

In [41]:
###### converting transaction_date and membership_date to datetime object
transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'], format='%Y%m%d')
transactions_df['membership_expire_date'] = pd.to_datetime(transactions_df['membership_expire_date'], format='%Y%m%d')

# removing duplicates and leaving only the latest transaction date
transactions_df_lt = transactions_df.loc[transactions_df.groupby('msno').transaction_date.idxmax()]

# creating remaining plan duration column
transactions_df_lt['remaining_plan_duration'] = transactions_df_lt['membership_expire_date'] - transactions_df_lt['transaction_date']

# creating discount column
transactions_df_lt['is_discount'] = transactions_df_lt.apply(lambda x: '0' if (x['actual_amount_paid'] -x['plan_list_price'])>=0 else '1', axis=1)

### User logs Data

In [42]:
# Converting date into datetime object
user_logs_df['date'] = pd.to_datetime(user_logs_df['date'], format='%Y%m%d')

#create new dataframe from transactions table with only msno and latest transaction date
latest_transactions_per_msno = transactions_df_lt[['msno', 'transaction_date']]

#Merge user logs with latest transaction date
merged_df = user_logs_df.merge(latest_transactions_per_msno, on='msno', how='left')

# drop msno's where transaction date is NaN
merged_df = merged_df.dropna(subset=['transaction_date'])

#removing rows where the user log data is before the last transaction date
user_logs_atd= merged_df.loc[merged_df['date']>=merged_df['transaction_date']]

# removing data column
user_logs_atd = user_logs_atd.drop(columns=['date','transaction_date'])

#groupby msno and summing all values
user_logs_atd = user_logs_atd.groupby('msno').sum()

### Merging

In [43]:
# Merging transactions, user logs and members data to Train dataframe
test_df = test_df.merge(transactions_df_lt, on='msno', how='left')
test_df = test_df.merge(user_logs_atd, on='msno', how='left')
test_df = test_df.merge(members_df, on='msno', how='left')
test_df = test_df.merge(trendline_df, on='msno', how='left')

### Feature engineering

In [47]:
# Drop Gender column
test_df = test_df.drop(['gender'],axis=1)

In [48]:
# Drop NaN
test_df = test_df.dropna()

In [49]:
# Changing bd outliers to NaN
test_df['bd'] = test_df['bd'].apply(lambda x: np.nan if x <14 or x > 75 else x)

In [50]:
# Converting remaining plan duration to int
test_df['remaining_plan_duration'] = test_df['remaining_plan_duration'].dt.days

In [51]:
# Converting total seconds to hours
test_df['total_secs']= round(test_df['total_secs']/3600,2)

In [52]:
# Creating average usage (hours) per day from latest transaction
test_df['usage_from_ltd'] = round(test_df['total_secs']/test_df['remaining_plan_duration'],2)
# replacing Nan with 0 
test_df['usage_from_ltd'].replace([np.nan], 0, inplace=True)
# replacing inf values with the totalsecs usage.
test_df['usage_from_ltd'] = np.where(test_df['usage_from_ltd'] == np.inf, test_df['total_secs'],test_df['usage_from_ltd'])

In [53]:
# converting registration time to datetime object
test_df['registration_init_time'] = pd.to_datetime(test_df['registration_init_time'], format='%Y%m%d')

In [54]:
# creating discount percentage column
test_df['discount_percentage'] = round((test_df['plan_list_price'] - test_df['actual_amount_paid'])/test_df['plan_list_price'],2)

test_df['discount_percentage'].replace([np.nan,-0.01,0.01], 0, inplace=True)

### Cyclic Encoding

In [55]:
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [56]:
# cyclic encoding transaction date
#year
test_df['last_transaction_year'] = test_df['transaction_date'].dt.year

#month
test_df['last_transaction_month'] = test_df['transaction_date'].dt.month
test_df = encode(test_df, 'last_transaction_month', 12)

#day
test_df['last_transaction_day'] = test_df['transaction_date'].dt.day
test_df= encode(test_df, 'last_transaction_day', 31)

test_df =test_df.drop(['last_transaction_month', 'last_transaction_day'],axis=1)

In [57]:
# cyclic encoding membership expire date
#year
test_df['expire_year'] = test_df['membership_expire_date'].dt.year

#month
test_df['expire_month'] = test_df['membership_expire_date'].dt.month
test_df = encode(test_df, 'expire_month', 12)

#day
test_df['expire_day'] = test_df['membership_expire_date'].dt.day
test_df= encode(test_df, 'expire_day', 31)

# droping month and day columns
test_df =test_df.drop(['expire_month', 'expire_day'],axis=1)

In [58]:
# cyclic encoding registration date
#year
test_df['registration_year'] = test_df['registration_init_time'].dt.year

#month
test_df['registration_month'] = test_df['registration_init_time'].dt.month
test_df = encode(test_df, 'registration_month', 12)

#day
test_df['registration_day'] = test_df['registration_init_time'].dt.day
test_df= encode(test_df, 'registration_day', 31)

# droping month and day columns
test_df =test_df.drop(['registration_month', 'registration_day'],axis=1)


In [59]:
# droping date columns to avoid data leakage
test_df =test_df.drop(['registration_init_time', 'membership_expire_date', 'transaction_date'],axis=1)

In [60]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 568823 entries, 2 to 1076940
Data columns (total 43 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   msno                        568823 non-null  object 
 1   is_churn                    568823 non-null  int64  
 2   payment_method_id           568823 non-null  float64
 3   payment_plan_days           568823 non-null  float64
 4   plan_list_price             568823 non-null  float64
 5   actual_amount_paid          568823 non-null  float64
 6   is_auto_renew               568823 non-null  float64
 7   is_cancel                   568823 non-null  float64
 8   remaining_plan_duration     568823 non-null  int64  
 9   is_discount                 568823 non-null  object 
 10  num_25                      568823 non-null  float64
 11  num_50                      568823 non-null  float64
 12  num_75                      568823 non-null  float64
 13  num_985           

In [61]:
test_df.value_counts('is_churn')

is_churn
0    568823
Name: count, dtype: int64

## export not preprocessed test_df to csv

In [62]:
# Export to CSV
test_df.to_csv('test.csv', index=False)

## preprocess test_df

In [75]:
#dropping invaluable columns based on feature importance (see model notebooks)
test_df = test_df.drop(['msno', 'bd', 'payment_method_id', 'city', 'registered_via'], axis=1)

KeyError: "['msno', 'bd', 'payment_method_id', 'city', 'registered_via'] not found in axis"

In [70]:
#list features for their scalers
robust_features = [               'remaining_plan_duration', 
          'usage_from_ltd',
              'payment_plan_days',
              'plan_list_price',
              'actual_amount_paid',
             'num_50',
              'num_75',
              'num_985',
              
              'expire_year',
              'last_transaction_year'
             ]
minmax_features = ['registration_year']
normal_features = [ 'num_25',
              
              'num_100',
              'num_unq',
              'total_secs',]

In [71]:
# Build the pipeline with the different steps, this pipeline does not include the model
robust_pipeline = make_pipeline(RobustScaler())
minmax_pipeline = make_pipeline(MinMaxScaler())
log_pipeline = make_pipeline( 
    FunctionTransformer(np.log1p, validate=True),
    StandardScaler())

preprocessor_new = ColumnTransformer(
    transformers=[
        ('Robust', robust_pipeline, robust_features),
        ('MinMax', minmax_pipeline, minmax_features),
        ('Log', log_pipeline, normal_features)
    ], remainder='passthrough'
)
    

In [72]:
preprocessor_new

ColumnTransformer(remainder='passthrough',
                  transformers=[('Robust',
                                 Pipeline(steps=[('robustscaler',
                                                  RobustScaler())]),
                                 ['remaining_plan_duration', 'usage_from_ltd',
                                  'payment_plan_days', 'plan_list_price',
                                  'actual_amount_paid', 'num_50', 'num_75',
                                  'num_985', 'expire_year',
                                  'last_transaction_year']),
                                ('MinMax',
                                 Pipeline(steps=[('minmaxscaler',
                                                  MinMaxScaler())]),
                                 ['registration_year']),
                                ('Log',
                                 Pipeline(steps=[('functiontransformer',
                                                  FunctionTransformer(func=<ufunc 'log1p'>,
                                                                      validate=True)),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['num_25', 'num_100', 'num_unq',
                                  'total_secs'])])

In [79]:
#drop churn column from df
test_df = test_df.drop(['is_churn'], axis=1)


KeyError: "['is_churn'] not found in axis"

In [78]:
# Use fit on test_df and then transform
preprocessor_new.fit(test_df)
test_df_transformed = preprocessor_new.transform(test_df)

In [82]:
# Convert transformed data back to DataFrame to ensure column order is maintained
test_df_transformed = pd.DataFrame(
    test_df_transformed, 
    columns=test_df.columns
)

## split transformed test_df into three and export CSVs

In [84]:
# Calculate split sizes
rows = len(test_df_transformed)
size = rows // 3  # Integer division

# Split the DataFrame
test_transformed_df1 = test_df_transformed.iloc[:size]
test_transformed_df2 = test_df_transformed.iloc[size:2*size]
test_transformed_df3 = test_df_transformed.iloc[2*size:]


In [85]:
# Export to CSV
test_transformed_df1.to_csv('test_transformed_df1.csv', index=False)
test_transformed_df2.to_csv('test_transformed_df2.csv', index=False)
test_transformed_df3.to_csv('test_transformed_df3.csv', index=False)